In [ ]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json backoff pyyaml facebook_business pystache

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from environs import Env

from adopt.malaria import get_confs, get_df, load_basics, window, get_db_conf
from adopt.marketing import Marketing
from adopt.facebook.state import CampaignState, get_api
from adopt.malaria import load_typed_json
from adopt.marketing import AudienceConf, CampaignConf
from adopt.campaign_queries import get_campaigns_for_user, create_campaign_confs, create_campaign_for_user
from typing import List
import json

env = Env()
env.read_env('.env-upswell-vlab', override=True)
db_conf = get_db_conf(env)

In [6]:
USER = "malaria2@vlab.digital"
CAMPAIGN = "mnm-jan-2021"
CAMPAIGNID = next(c['id'] for c in get_campaigns_for_user(USER, db_conf) 
                  if c['name'] == CAMPAIGN)

# create_campaign_for_user(USER, CAMPAIGN, db_conf)

In [37]:
userinfo, config, db_conf, state, m, confs = load_basics(CAMPAIGNID, env)

In [38]:
images = {i['name']: i for i in state.account.get_ad_images(fields=['name', 'hash'])}

# CREATING CONFS - MNM

In [79]:
# you have 10650 to spend

(3000000.0 / 7300)

410.958904109589

In [75]:
c = {'optimization_goal': 'REPLIES',
     'destination_type': 'MESSENGER',
     'adset_hours': 48,
     'budget': 3000000.0,
     'min_budget': 10000.0,
     'opt_window': 20*24,
     'end_date': '2021-03-29',
     'page_id': '102998371752603',
     'instagram_id': None,
     'proportional': False,
     'ad_account': '2240415876083895',
     'ad_campaign': 'impact-evaluation-1shot-global-insights'}

config = CampaignConf(**c)

create_campaign_confs(CAMPAIGNID, "opt", [config._asdict()], db_conf)

('e49ba210-9928-4d9f-b62a-874c5652bb71',
 datetime.datetime(2021, 2, 23, 10, 7, 13, 40204, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
 'opt',
 None,
 [{'ad_account': '2240415876083895',
   'ad_campaign': 'impact-evaluation-1shot-global-insights',
   'adset_hours': 48,
   'budget': 3000000.0,
   'destination_type': 'MESSENGER',
   'end_date': '2021-03-29',
   'instagram_id': None,
   'min_budget': 10000.0,
   'opt_window': 480,
   'optimization_goal': 'REPLIES',
   'page_id': '102998371752603',
   'proportional': False}])

In [ ]:
# select current_state, fb_error_code, state_json->'previousOutput' from states where current_form = 'extrabasehin' and timeout_date is not null;

# select timeout_date, current_state, fb_error_code, state_json->'tokens'->>0, state_json->'previousOutput'->>'token', state_json->'error', state_json->'previousOutput' from states where current_form = 'extrabasehin' and fb_error_code = '2022' and state_json->'previousOutput'->>'token' is not null;

# select fb_error_code, count(*) from states where current_form = 'extrabasehin' and (state_json->'previousOutput'->>'token' is not null or state_json->'tokens'->>0 is not null) group by fb_error_code;

In [ ]:
import json
from adopt.marketing import make_audience_conf, dict_from_nested_type

audience_confs = [
  {
    "name": "vlab-mnm-kutcha",
    "shortcodes": ["extrabasehin1shot"],
    "subtype": "LOOKALIKE",
    "lookalike": {
      "name": "vlab-mnm-kutcha-lookalike",
      "target": 2000,
      "spec": {
        "country": "IN",
        "starting_ratio": 0.0,
        "ratio": 0.2
      }
    },
    "question_targeting": {
      "op": "equal",
      "vars": [
        {"type": "response", "value": "dwelling"},
        {"type": "constant", "value": "A"}
      ]
    }
  },
  {
    "name": "vlab-mnm-respondents-1shot-global-insights",
    "shortcodes": ["extrabasehin1shot"],
    "subtype": "CUSTOM"
  }
]



confs = [dict_from_nested_type(a) for a in audience_confs]

create_campaign_confs(CAMPAIGNID, "audience", confs, db_conf)

In [ ]:
from adopt.marketing import CreativeConf

def _creative_conf(name, image, body, form):
    return {
        "name": name,
        "image": image['name'],
        "image_hash": image['hash'],
        "body": body,
        "welcome_message": body,
        "link_text": "Messenger main survey shuroo karen!",
        "button_text": "Start!",
        "form": form,
    }

image_confs = [
    ('vlab-mnm-mother-daughter-voice-be-heard', '1.png', 'अपने विचार दुनिया को बताएं!'),
    ('vlab-mnm-family-flowers-keep-healthy', '2.png', 'हमें अपने स्वस्थ जीवन के राज बताएं और फ्री डाटा क्रेडिट जीतें।'),
    # ('vlab-mnm-schoolgirl-laptop-got-10-min', '3.png', 'क्या आपके पास सिर्फ १० मिनिट हैं? मोबाइल डाटा क्रेडिट में २०० रुपये पाएँ। आपकी राय हमारे लिए महत्वपूर्ण है!'),
    ('vlab-mnm-grandfather-bike-healthy', '4.png', 'अपने कम्यूनिटी को स्वस्थ रखने में हमें मदद करें- हमें बताएं आप क्या सोचते है!'),
    ('vlab-mnm-woman-sari-texting-got-time', '5.png', 'एक छोटे से सर्वे के लिए आपके पास टाइम है?'),
]

nncreatives = [CreativeConf(**_creative_conf(name, images[n], body, 'extrabasehin1shot')) for name, n, body in image_confs]
# creatives = load_typed_json("config/creatives.json", List[CreativeConf])
confs = [c._asdict() for c in creatives]

create_campaign_confs(CAMPAIGNID, "creative", confs, db_conf)

In [42]:
import pandas as pd
from facebook_business.adobjects.targetinggeolocationcustomlocation \
    import TargetingGeoLocationCustomLocation
from facebook_business.adobjects.targetinggeolocationcity import TargetingGeoLocationCity
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from facebook_business.adobjects.targeting import Targeting

from adopt.marketing import make_stratum_conf

def load_cities(path):
    cities = pd.read_csv(path)
    cities = cities[cities.rad >= 1.0]
    return cities

def create_location(lat, lng, rad):
    return {
        TargetingGeoLocationCustomLocation.Field.latitude: lat,
        TargetingGeoLocationCustomLocation.Field.longitude: lng,
        TargetingGeoLocationCustomLocation.Field.radius: rad,
        TargetingGeoLocationCustomLocation.Field.distance_unit: 'kilometer',
    }

def make_stratum(facebook_targeting, quota, df):
    locs = [create_location(r.lat, r.lng, r.rad) 
           for _, r in df.iterrows()]

    targeting = {
        **facebook_targeting,
        Targeting.Field.geo_locations: {
            TargetingGeoLocation.Field.location_types: ['home'],
            TargetingGeoLocation.Field.custom_locations: locs
        },
    }

    id_ = df.disthash.iloc[0]

    conf = { 'metadata': {'stratumid': id_},
             'facebook_targeting': targeting, 
             'creatives': ['vlab-mnm-mother-daughter-voice-be-heard',
                           'vlab-mnm-family-flowers-keep-healthy',
                           # 'vlab-mnm-schoolgirl-laptop-got-10-min',
                           'vlab-mnm-grandfather-bike-healthy',
                           'vlab-mnm-woman-sari-texting-got-time'],
             'shortcodes': ['extrabasehin', 'extrabasehin1shot']}
    

    base = {**conf, 
            'id': id_,
            'quota': int(quota * 0.7), 
            'audiences': [],
            'excluded_audiences': ["vlab-mnm-respondents-panel", "vlab-mnm-respondents-round-B", "vlab-mnm-kutcha-lookalike", "vlab-mnm-respondents-1shot", "vlab-mnm-respondents-1shot-global-insights"], 
            'question_targeting': {
                'op': 'and',
                'vars': [
                    { 'op': 'equal',
                      'vars': [{'type': 'response', 'value': 'md:stratumid'},
                               {'type': 'constant', 'value': id_}]},
                    { 'op': 'not_equal',
                      'vars': [{'type': 'response', 'value': 'dwelling'},
                               {'type': 'constant', 'value': 'A'}]}
                ]
            }}

    kutcha = {**conf, 
              'id': id_ + '-kutcha',
              'quota': int(quota * 0.3), 
              'audiences': ["vlab-mnm-kutcha-lookalike"],
              'excluded_audiences': ["vlab-mnm-respondents-panel", "vlab-mnm-respondents-round-B", "vlab-mnm-respondents-1shot", "vlab-mnm-respondents-1shot-global-insights"], 
              'question_targeting': {
                  'op': 'and',
                  'vars': [
                      { 'op': 'equal',
                        'vars': [{'type': 'response', 'value': 'md:stratumid'},
                                 {'type': 'constant', 'value': id_}]},
                      { 'op': 'equal',
                        'vars': [{'type': 'response', 'value': 'dwelling'},
                                 {'type': 'constant', 'value': 'A'}]}
                  ]
              }}

    return [base, kutcha]

In [ ]:
from adopt.marketing import dict_from_nested_type

df = load_cities('output/cities.csv')
df = df[['disthash', 'rad', 'lng', 'lat', 'creative_group', 'include_audience']]
strata = df.groupby('disthash').apply(lambda df: make_stratum({'age_min': 20}, 60, df)).tolist()

strata_data = [dict_from_nested_type(make_stratum_conf(c)) for s in strata for c in s]

create_campaign_confs(CAMPAIGNID, "stratum", strata_data, db_conf)

# TESTING

In [80]:
mal = load_basics(CAMPAIGNID, env)

In [ ]:
from adopt.malaria import update_audience_for_campaign, update_ads_for_campaign

instructions, report = update_ads_for_campaign(mal)

In [ ]:
instructions, _ = update_audience_for_campaign(mal)

In [ ]:
from adopt.malaria import run_instructions

run_instructions(instructions, mal.state)

In [17]:
mal.confs['opt'][0].budget / 7300 - mal.state.total_spend / 73

4210.190821917808

In [ ]:
from adopt.malaria import hydrate_strata, hydrate_audiences, get_df

userinfo, config, db_conf, state, m, confs = mal

# df = get_df(db_conf, userinfo.survey_user, audience_confs)

strata = hydrate_strata(state, confs["stratum"], confs["creative"])
df = get_df(db_conf, userinfo.survey_user, strata)

In [27]:
audience_confs = confs["audience"]
audiences = hydrate_audiences(df, m, audience_confs)

In [ ]:
strata = hydrate_strata(state, confs["stratum"], confs["creative"])
df = get_df(db_conf, userinfo.survey_user, strata)

In [81]:
import pandas as pd
from adopt.campaign_queries import get_last_adopt_report


rdf = pd.DataFrame(get_last_adopt_report(CAMPAIGNID, "FACEBOOK_ADOPT", mal.db_conf)).T

In [82]:
from adopt.malaria import days_left

rdf['expected'] = (rdf.respondents + ((rdf.budget / rdf.price) * days_left(mal.config))).astype(int)
rdf['expected_share'] = rdf.expected / rdf.expected.sum()

In [83]:
mal.state.total_spend / 73

2994.571095890411

In [84]:
days_left(mal.config) * rdf.budget.sum() / 7300

13551.477123287672

In [85]:
rdf.expected.sum()

5394

In [86]:
rdf.respondents.sum()

4882.0

In [87]:

with pd.option_context("display.max_rows", 160, "display.max_columns", 10):
    display(rdf.sort_values('expected'))

,budget,goal,price,remaining,respondents,spend,expected,expected_share
e5d2d035,63818.0,42.0,9.789860e+05,35.0,7.0,489493.0,9,0.001669
fb794055-kutcha,63818.0,18.0,6.761360e+05,11.0,7.0,338068.0,10,0.001854
c25e6958-kutcha,63818.0,18.0,3.814560e+05,12.0,6.0,190728.0,11,0.002039
5b788486-kutcha,63818.0,18.0,2.602540e+05,14.0,4.0,260254.0,12,0.002225
2568806d-kutcha,63818.0,18.0,2.763960e+05,11.0,7.0,276396.0,14,0.002595
d56fd7da-kutcha,63818.0,18.0,3.317000e+05,10.0,8.0,165850.0,14,0.002595
c5524846-kutcha,63818.0,18.0,4.946080e+05,8.0,10.0,247304.0,14,0.002595
b406895f-kutcha,63818.0,18.0,3.433000e+05,9.0,9.0,171650.0,15,0.002781
0d1e3ccb-kutcha,63818.0,18.0,2.969100e+05,10.0,8.0,148455.0,15,0.002781
9a766899-kutcha,63818.0,18.0,3.727960e+05,7.0,11.0,186398.0,16,0.002966


In [35]:
import numpy as np

np.linalg.norm(rdf.expected_share - rdf.goal)

0.03000335994283512

In [36]:
np.linalg.norm(rdf.respondent_share - rdf.goal)

0.03685672725643937

In [99]:
t = (5000 / 80)

t*.7, t*.3

(43.75, 18.75)

In [97]:
rdf.expected.sum()

7252.580094737863

In [70]:
report.respondents.sum()

11669.0

In [ ]:
df.question_ref

In [78]:
rdf = pd.DataFrame(report).T

In [ ]:
rdf